<a href="https://colab.research.google.com/github/peculab/AI4JUBO/blob/main/JuboDeath_V8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#訓練資料是 mortality_2020_2023_0910/training_data_v2_0916
#外部驗證資料是 mortality_2024_0910/external_validation_v2_0916

#次族群

- <= 85 & > 85
- ADL 變好 & ADL 變差
- 男性 & 女性

In [1]:
!pip install shap plotly xgboost --quiet

In [2]:
!pip uninstall shap -y
!pip install shap --no-deps

Found existing installation: shap 0.48.0
Uninstalling shap-0.48.0:
  Successfully uninstalled shap-0.48.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.3 MB/s eta 0:00:00


In [3]:
!pip install ace_tools

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, mean_absolute_error, r2_score
)

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

外部資料讀入

In [45]:
# read data and put it in a dataframe
# 在 google 工作表載入外部資料 gsheets

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1jXuepVDDrnhOKYMqy-RQKeQlWr7atvKdeq_Y0PosMDM/edit?usp=sharing')
worksheet = gsheets.worksheet("external_validation_v2_0916")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
external = pd.DataFrame(worksheet)
external = external.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,first_ 意識總分,last_ 意識總分,意識總分Max,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall
0,1376,NaN,NaN,NaN,0,197,1,77,0,90,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
1,1341,NaN,NaN,NaN,1,77,1,78,0,10,...,10.0,7.0,10.0,-3.0,NaN,0.0,0.0,0.0,0.0,0.0
2,1354,NaN,NaN,NaN,1,6,1,73,1,20,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
3,1351,NaN,NaN,NaN,1,3,1,74,1,5,...,14.0,14.0,14.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,1326,NaN,NaN,NaN,1,32,0,93,1,0,...,9.0,9.0,9.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0


In [48]:
external.columns

Index(['H01_NUM', 'dbname', '入家日期', '結案日期', '死亡標記', '觀察天數', '性別_is_male',
       '預估年齡', 'DNR_flag', 'ADL_總分_max', 'ADL_first_score', 'ADL_last_score',
       'ADL_diff_seq', 'ADL_std', 'ADL_Max', 'ADL_Min', 'ADL_明顯惡化',
       'ADL_first_CouldNot', 'ADL_last_CouldNot', 'disease_first_score',
       'disease_last_score', 'disease_diff_seq', 'disease_std', 'disease_Max',
       'disease_Min', '六個月內住院次數', 'first_has_denture', 'last_has_denture',
       'diff_has_denture', 'first_ 意識_E', 'last_ 意識_E', 'first_ 意識_V',
       'last_ 意識_V', 'first_ 意識_M', 'last_ 意識_M', 'first_ 意識總分', 'last_ 意識總分',
       '意識總分Max', '意識總分_diff', '意識分級', '使用呼吸輔具', 'first_has_feeding_tube',
       'last_has_feeding_tube', 'diff_has_feeding_tube', 'had_fall', '呼吸_ADL低',
       '高齡_住院多'],
      dtype='object')

In [49]:
external['呼吸_ADL低'] = ((external['使用呼吸輔具'] == 1) & (external['ADL_總分_max'] < 30)).astype(int)
external['高齡_住院多'] = ((external['預估年齡'] > 85) & (external['六個月內住院次數'] > 2)).astype(int)
external['ADL低_明顯惡化'] = ((external['ADL_總分_max'] < 30) & (external['ADL_明顯惡化'] == 1)).astype(int)
external['DNR_呼吸'] = ((external['DNR_flag'] == 1) & (external['使用呼吸輔具'] == 1)).astype(int)
external['超級重症'] = (
    (external['DNR_flag'] == 1) &
    (external['使用呼吸輔具'] == 1) &
    (external['ADL_總分_max'] < 30) &
    (external['六個月內住院次數'] > 1)
).astype(int)
external["low_adl_no_var"] = ((external["ADL_std"].isnull()) & (external["ADL_總分_max"] < 5)).astype(int)

In [50]:
external.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,6858.0,1397.165063,786.105289,4.0,1184.0,1266.0,1390.000000,20295.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,6858.0,0.266258,0.442033,0.0,0.0,0.0,1.000000,1.000000
觀察天數,6858.0,214.050160,108.630929,0.0,148.0,239.0,301.000000,365.000000
性別_is_male,6858.0,0.501896,0.500033,0.0,0.0,1.0,1.000000,1.000000
預估年齡,6858.0,78.615194,11.752227,1.0,72.0,81.0,87.000000,124.000000
DNR_flag,6858.0,0.467192,0.498959,0.0,0.0,0.0,1.000000,1.000000
ADL_總分_max,6858.0,30.230388,31.799113,0.0,0.0,20.0,50.000000,100.000000


In [51]:
ex_missing_info = external.isnull().sum().to_frame(name='Missing Count')
ex_missing_info['Missing Ratio'] = (ex_missing_info['Missing Count'] / len(external)).round(4)
ex_missing_info = ex_missing_info.sort_values(by='Missing Ratio', ascending=True)
ex_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_Max,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_Min,0,0.0000
ADL_last_CouldNot,0,0.0000


訓練資料讀入

In [52]:
# read data and put it in a dataframe
# 在 google 工作表載入訓練資料 gsheets

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1_D_ryADEhFBC0322zwlI7uQZz5xELWlZ-3CO549cLS0/edit?usp=sharing')
worksheet = gsheets.worksheet("training_data_v2_0916")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
df = pd.DataFrame(worksheet)
df = df.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
df.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,first_ 意識總分,last_ 意識總分,意識總分Max,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall
0,1325,NaN,NaN,NaN,0,739,1,66,1,0,...,13.0,12.0,13.0,-1.0,NaN,0.0,0.0,0.0,0.0,0.0
1,1160,NaN,NaN,NaN,0,788,1,89,1,95,...,15.0,13.0,15.0,-2.0,NaN,0.0,0.0,0.0,0.0,0.0
2,1253,NaN,NaN,NaN,0,1292,0,89,0,60,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
3,1342,NaN,NaN,NaN,0,584,0,93,1,30,...,14.0,14.0,14.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,1343,NaN,NaN,NaN,0,583,1,90,0,90,...,15.0,15.0,15.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0


In [53]:
df['呼吸_ADL低'] = ((df['使用呼吸輔具'] == 1) & (df['ADL_總分_max'] < 30)).astype(int)
df['高齡_住院多'] = ((df['預估年齡'] > 85) & (df['六個月內住院次數'] > 2)).astype(int)
df['ADL低_明顯惡化'] = ((df['ADL_總分_max'] < 30) & (df['ADL_明顯惡化'] == 1)).astype(int)
df['DNR_呼吸'] = ((df['DNR_flag'] == 1) & (df['使用呼吸輔具'] == 1)).astype(int)
df['超級重症'] = (
    (df['DNR_flag'] == 1) &
    (df['使用呼吸輔具'] == 1) &
    (df['ADL_總分_max'] < 30) &
    (df['六個月內住院次數'] > 1)
).astype(int)
df["low_adl_no_var"] = ((df["ADL_std"].isnull()) & (df["ADL_總分_max"] < 5)).astype(int)

In [54]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,30324.0,1562.316119,4683.787577,1.0,1162.0,1225.00000,1323.000000,100463.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,30324.0,0.177780,0.382334,0.0,0.0,0.00000,0.000000,1.000000
觀察天數,30324.0,664.162841,439.423367,0.0,278.0,647.00000,1046.000000,1460.000000
性別_is_male,30324.0,0.482060,0.499686,0.0,0.0,0.00000,1.000000,1.000000
預估年齡,30324.0,79.314602,11.954595,0.0,72.0,82.00000,88.000000,125.000000
DNR_flag,30324.0,0.415084,0.492745,0.0,0.0,0.00000,1.000000,1.000000
ADL_總分_max,30324.0,32.127853,33.075378,0.0,0.0,20.00000,55.000000,100.000000


In [55]:
df_missing_info = df.isnull().sum().to_frame(name='Missing Count')
df_missing_info['Missing Ratio'] = (df_missing_info['Missing Count'] / len(df)).round(4)
df_missing_info = df_missing_info.sort_values(by='Missing Ratio', ascending=True)
df_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_Max,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_Min,0,0.0000
ADL_last_CouldNot,0,0.0000


In [56]:
features = df_missing_info[df_missing_info['Missing Ratio']<0.3].index.tolist()

In [57]:
features

['H01_NUM',
 '性別_is_male',
 '觀察天數',
 '死亡標記',
 '預估年齡',
 'ADL_Max',
 'ADL_總分_max',
 'DNR_flag',
 'ADL_Min',
 'ADL_last_CouldNot',
 'ADL_first_CouldNot',
 'ADL_明顯惡化',
 '六個月內住院次數',
 '高齡_住院多',
 '呼吸_ADL低',
 'ADL低_明顯惡化',
 'DNR_呼吸',
 '超級重症',
 'low_adl_no_var',
 'ADL_first_score',
 'ADL_last_score',
 'ADL_diff_seq',
 'diff_has_denture',
 'last_has_denture',
 '意識總分Max',
 '使用呼吸輔具',
 'first_has_denture',
 'had_fall',
 'diff_has_feeding_tube',
 'last_has_feeding_tube',
 'first_has_feeding_tube',
 'ADL_std']

In [58]:
dfNew = df[features]

In [59]:
dfNew = dfNew.fillna(0)

In [60]:
from sklearn.base import BaseEstimator, ClassifierMixin

class HybridXGBRF(BaseEstimator, ClassifierMixin):
    def __init__(self, xgb_model=None, rf_model=None, alpha=0.5):
        self.xgb_model = xgb_model
        self.rf_model = rf_model
        self.alpha = alpha
        self._init_models()

    def _init_models(self):
        # Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 800, 'subsample': 1.0}
        # "XGBClassifier": XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state=42, eval_metric='logloss'),

        self.xgb = self.xgb_model or XGBClassifier(
            eval_metric="logloss",
            random_state=42,
            colsample_bytree=0.8,     # ✅ 降低每棵樹看到的特徵比例 → 提高多樣性
            learning_rate=0.01,       # ✅ 稍微提升學習率搭配更早停止
            max_depth=5,              # ✅ 降低單棵樹複雜度 → 降低過擬合
            n_estimators=200,         # ✅ 總樹數可略減以免累積錯誤
            subsample=1.0,            # ✅ 樣本隨機抽樣 → 提升隨機性
            verbosity=0,
            use_label_encoder=False
        )
        self.rf = self.rf_model or RandomForestClassifier(
            n_estimators=100,
            random_state=42
        )

    def fit(self, X, y):
        self._init_models()  # 每次 fit 要重設模型
        self.xgb.fit(X, y)
        self.rf.fit(X, y)
        return self

    def predict_proba(self, X):
        xgb_prob = self.xgb.predict_proba(X)[:, 1]
        rf_prob = self.rf.predict_proba(X)[:, 1]
        blended = self.alpha * xgb_prob + (1 - self.alpha) * rf_prob
        return np.vstack([1 - blended, blended]).T

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] > 0.5).astype(int)

    def get_params(self, deep=True):
        return {
            'xgb_model': self.xgb_model,
            'rf_model': self.rf_model,
            'alpha': self.alpha
        }

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        self._init_models()  # 重新初始化模型
        return self

In [61]:
# 要移除的欄位，是代表身分標記，以及天數
drop_columns = ['H01_NUM', '觀察天數']

# 丟掉這些欄位
dfNew = dfNew.drop(columns=drop_columns)

In [62]:
dfNew

,性別_is_male,死亡標記,預估年齡,ADL_Max,ADL_總分_max,DNR_flag,ADL_Min,ADL_last_CouldNot,ADL_first_CouldNot,ADL_明顯惡化,...,diff_has_denture,last_has_denture,意識總分Max,使用呼吸輔具,first_has_denture,had_fall,diff_has_feeding_tube,last_has_feeding_tube,first_has_feeding_tube,ADL_std
0,1,0,66,0,0,1,0,0,0,0,...,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,1,0,89,95,95,1,10,0,0,1,...,-1.0,0.0,15.0,0.0,1.0,0.0,0.0,0.0,0.0,49.074773
2,0,0,89,60,60,0,30,0,0,1,...,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,16.431677
3,0,0,93,30,30,1,10,0,0,1,...,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,10.350983
4,1,0,90,90,90,0,5,0,0,1,...,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,45.434411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30319,1,0,48,100,100,0,100,0,0,0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
30320,1,0,41,100,100,0,100,0,0,0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
30321,0,0,60,100,100,0,100,0,0,0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
30322,0,0,54,100,100,0,100,0,0,0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [63]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    confusion_matrix, roc_curve, auc, classification_report
)
import plotly.graph_objects as go
import plotly.express as px

In [64]:
# === 資料準備 ===
X = dfNew.drop(columns=['死亡標記'])
y = df['死亡標記']

In [65]:
X_missing_info = X.isnull().sum().to_frame(name='Missing Count')
X_missing_info['Missing Ratio'] = (X_missing_info['Missing Count'] / len(X)).round(4)
X_missing_info = X_missing_info.sort_values(by='Missing Ratio', ascending=True)
X_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
預估年齡,0,0.0
ADL_Max,0,0.0
ADL_總分_max,0,0.0
DNR_flag,0,0.0
ADL_Min,0,0.0
ADL_last_CouldNot,0,0.0
ADL_first_CouldNot,0,0.0
ADL_明顯惡化,0,0.0
六個月內住院次數,0,0.0


# 開始進行訓練

In [66]:
X.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,30324.0,0.482060,0.499686,0.0,0.0,0.0,1.000000,1.000000
預估年齡,30324.0,79.314602,11.954595,0.0,72.0,82.0,88.000000,125.000000
ADL_Max,30324.0,32.127853,33.075378,0.0,0.0,20.0,55.000000,100.000000
ADL_總分_max,30324.0,32.127853,33.075378,0.0,0.0,20.0,55.000000,100.000000
DNR_flag,30324.0,0.415084,0.492745,0.0,0.0,0.0,1.000000,1.000000
ADL_Min,30324.0,19.707492,28.053787,0.0,0.0,5.0,30.000000,100.000000
ADL_last_CouldNot,30324.0,0.018962,0.136393,0.0,0.0,0.0,0.000000,1.000000
ADL_first_CouldNot,30324.0,0.016983,0.129211,0.0,0.0,0.0,0.000000,1.000000
ADL_明顯惡化,30324.0,0.207822,0.405756,0.0,0.0,0.0,0.000000,1.000000
六個月內住院次數,30324.0,2.897342,10.811437,0.0,0.0,1.0,3.000000,618.000000


In [67]:
!pip install lifelines

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lifelines import CoxPHFitter
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV

# 假設 HybridXGBRF 已定義
all_models = {
    #"HybridXGBRF (Our Approach)": HybridXGBRF(alpha=1),
    "HybridXGBRF (Our Approach)": XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state=42, eval_metric='logloss', subsample=1.0, verbosity=0),
    "LogisticRegression (max_iter=200)": LogisticRegression(max_iter=200),
    "XGBClassifier": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=3, random_state=42, eval_metric='logloss'),
    "RandomForestClassifier": RandomForestClassifier(n_estimators=100, random_state=42),
    "LogisticRegression (max_iter=1000)": LogisticRegression(max_iter=1000),

    # 🔽 新增未測試模型
    "Ridge": make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', solver='saga', max_iter=1000, random_state=42)),
    "Lasso": make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=42)),
    "Elastic": make_pipeline(StandardScaler(), LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, max_iter=1000, random_state=42)),
}

In [69]:
import copy
from sklearn.base import clone
from sklearn.impute import SimpleImputer
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted

# Prepare CV and ROC Figure
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fig_roc = go.Figure()
mean_fpr = np.linspace(0, 1, 100)

results = []
trained_models = {}

for model_name, model in all_models.items():
    print(f"▶ Running CV for: {model_name}")
    accs, precs, recalls, f1s, aucs = [], [], [], [], []
    all_cm = np.zeros((2, 2), dtype=int)
    tprs = []

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model_fold = clone(model)

        try:
            # 嘗試直接 fit（若模型支援 NaN，會成功）
            model_fold.fit(X_train, y_train)

        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️ Missing value detected for {model_name} (fold {fold+1}) — applying median imputation.")

                imputer = SimpleImputer(strategy='median')
                X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X.columns)
                X_test = pd.DataFrame(imputer.transform(X_test), columns=X.columns)

                model_fold.fit(X_train, y_train)
            else:
                raise e  # 若是其他錯誤就直接拋出

        y_pred = model_fold.predict(X_test)
        y_prob = model_fold.predict_proba(X_test)[:, 1]

        # ROC
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0] = 0.0
        tprs.append(tpr_interp)
        aucs.append(roc_auc)

        # Metrics
        accs.append(accuracy_score(y_test, y_pred))
        precs.append(precision_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
        all_cm += confusion_matrix(y_test, y_pred)

    trained_models[model_name] = copy.deepcopy(model_fold)

    # ROC Curve
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)

    fig_roc.add_trace(go.Scatter(
        x=mean_fpr, y=mean_tpr, mode='lines',
        name=f"{model_name} (mean AUC={mean_auc:.3f})"
    ))

    results.append({
        'Model': model_name,
        'Accuracy Mean': np.mean(accs),
        'Accuracy Std': np.std(accs),
        'Precision Mean': np.mean(precs),
        'Recall Mean': np.mean(recalls),
        'F1 Score Mean': np.mean(f1s),
        'ROC AUC Mean': np.mean(aucs),
        'ROC AUC Std': np.std(aucs),
        'Confusion Matrix': all_cm
    })

# Add Random Baseline
fig_roc.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1], mode='lines',
    line=dict(dash='dash'), name='Random Baseline'
))

fig_roc.update_layout(
    title="ROC Curve Comparison (Cross-Validation Mean)",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    width=800, height=600
)

fig_roc.show()

▶ Running CV for: HybridXGBRF (Our Approach)
▶ Running CV for: LogisticRegression (max_iter=200)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/

▶ Running CV for: XGBClassifier
▶ Running CV for: RandomForestClassifier
▶ Running CV for: LogisticRegression (max_iter=1000)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/

▶ Running CV for: Ridge
▶ Running CV for: Lasso


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



▶ Running CV for: Elastic


In [70]:
import shap

# 訓練模型（完整資料）
xgb_model = all_models["HybridXGBRF (Our Approach)"]
xgb_model.fit(X, y)

# 建立 SHAP 解釋器
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X)

# 計算平均 SHAP 值絕對值（作為重要性）
import numpy as np
shap_abs_mean = np.abs(shap_values).mean(axis=0)

In [71]:
import pandas as pd
import plotly.express as px

# 整理成 DataFrame 並排序
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Mean |SHAP Value|': shap_abs_mean
}).sort_values(by='Mean |SHAP Value|', ascending=False)

# 畫前 20 名
top_n = 20
fig_bar = px.bar(
    importance_df.head(top_n),
    x='Mean |SHAP Value|',
    y='Feature',
    orientation='h',
    title="🎯 Top SHAP Features by Mean |SHAP|",
)
fig_bar.update_layout(yaxis=dict(categoryorder='total ascending'))
fig_bar.show()

In [72]:
import plotly.graph_objects as go

for r in results:
    cm = r['Confusion Matrix']
    model_name = r['Model']

    z = cm
    x_labels = ['Predicted 0', 'Predicted 1']
    y_labels = ['Actual 0', 'Actual 1']

    fig_cm = go.Figure(data=go.Heatmap(
        z=z,
        x=x_labels,
        y=y_labels,
        colorscale='Blues',
        text=z,
        texttemplate="%{text}"
    ))

    fig_cm.update_layout(
        title=f"Confusion Matrix - {model_name}",
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
        width=500,
        height=500
    )

    fig_cm.show()

In [73]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="ROC AUC Mean", ascending=False).reset_index(drop=True)
results_df

,Model,Accuracy Mean,Accuracy Std,Precision Mean,Recall Mean,F1 Score Mean,ROC AUC Mean,ROC AUC Std,Confusion Matrix
0,HybridXGBRF (Our Approach),0.859748,0.003484,0.802015,0.280277,0.415312,0.854817,0.002300,"[[24560, 373], [3880, 1511]]"
1,XGBClassifier,0.860177,0.003596,0.797197,0.286212,0.421088,0.851826,0.002438,"[[24541, 392], [3848, 1543]]"
2,RandomForestClassifier,0.850218,0.002499,0.608256,0.442958,0.512515,0.838164,0.004771,"[[23394, 1539], [3003, 2388]]"
3,LogisticRegression (max_iter=1000),0.860012,0.003589,0.771701,0.301609,0.433653,0.827191,0.002837,"[[24453, 480], [3765, 1626]]"
4,Lasso,0.859880,0.003717,0.772801,0.299754,0.431904,0.826395,0.002665,"[[24459, 474], [3775, 1616]]"
5,Elastic,0.859781,0.003487,0.773578,0.298455,0.430683,0.825541,0.002742,"[[24463, 470], [3782, 1609]]"
6,Ridge,0.859847,0.003533,0.774881,0.298084,0.430489,0.824762,0.002791,"[[24467, 466], [3784, 1607]]"
7,LogisticRegression (max_iter=200),0.859550,0.003643,0.769411,0.299568,0.431162,0.822154,0.002378,"[[24450, 483], [3776, 1615]]"


# 測試外部資料在 XGBoost 模型下的結果

In [74]:
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall,呼吸_ADL低,高齡_住院多,ADL低_明顯惡化,DNR_呼吸,超級重症,low_adl_no_var
0,1376,NaN,NaN,NaN,0,197,1,77,0,90,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,1341,NaN,NaN,NaN,1,77,1,78,0,10,...,0.0,0.0,0.0,0.0,0,0,1,0,0,0
2,1354,NaN,NaN,NaN,1,6,1,73,1,20,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3,1351,NaN,NaN,NaN,1,3,1,74,1,5,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,1326,NaN,NaN,NaN,1,32,0,93,1,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1


In [75]:
ex_X = external[features].drop(columns=['死亡標記'])
ex_y = external['死亡標記']

In [76]:
ex_X = ex_X.fillna(0)

In [77]:
# 丟掉這些欄位
ex_X = ex_X.drop(columns=drop_columns)

In [78]:
ex_X.describe().T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,6858.0,0.501896,0.500033,0.0,0.0,1.0,1.0,1.000000
預估年齡,6858.0,78.615194,11.752227,1.0,72.0,81.0,87.0,124.000000
ADL_Max,6858.0,30.230388,31.799113,0.0,0.0,20.0,50.0,100.000000
ADL_總分_max,6858.0,30.230388,31.799113,0.0,0.0,20.0,50.0,100.000000
DNR_flag,6858.0,0.467192,0.498959,0.0,0.0,0.0,1.0,1.000000
ADL_Min,6858.0,21.604695,28.223145,0.0,0.0,10.0,35.0,100.000000
ADL_last_CouldNot,6858.0,0.010353,0.101228,0.0,0.0,0.0,0.0,1.000000
ADL_first_CouldNot,6858.0,0.010207,0.100520,0.0,0.0,0.0,0.0,1.000000
ADL_明顯惡化,6858.0,0.135462,0.342242,0.0,0.0,0.0,0.0,1.000000
六個月內住院次數,6858.0,2.151356,9.206643,0.0,0.0,1.0,2.0,581.000000


In [79]:
eX_missing_info = ex_X.isnull().sum().to_frame(name='Missing Count')
eX_missing_info['Missing Ratio'] = (eX_missing_info['Missing Count'] / len(ex_X)).round(4)
eX_missing_info = eX_missing_info.sort_values(by='Missing Ratio', ascending=False)
eX_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
預估年齡,0,0.0
ADL_Max,0,0.0
ADL_總分_max,0,0.0
DNR_flag,0,0.0
ADL_Min,0,0.0
ADL_last_CouldNot,0,0.0
ADL_first_CouldNot,0,0.0
ADL_明顯惡化,0,0.0
六個月內住院次數,0,0.0


In [80]:
from sklearn.impute import SimpleImputer
from sklearn.base import clone
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted
import plotly.subplots as sp

def evaluate_all_models_visual(models: dict, X_val, y_val):
    mean_fpr = np.linspace(0, 1, 100)
    fig_roc = go.Figure()
    results = []

    # 建立混淆矩陣子圖
    num_models = len(models)
    cols = 3
    rows = int(np.ceil(num_models / cols))

    fig_cm = sp.make_subplots(
        rows=rows, cols=cols,
        subplot_titles=list(models.keys()),
        horizontal_spacing=0.15,
        vertical_spacing=0.15
    )

    for i, (model_name, model) in enumerate(models.items()):
        print(f"🔍 Evaluating {model_name}...")

        # 嘗試使用原始資料
        X_input = X_val.copy()
        y_input = y_val

        # 若模型不支援 NaN，則補值
        try:
            # 嘗試呼叫 predict_proba
            _ = model.predict_proba(X_input)
        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️  {model_name} 不支援 NaN，自動補值中...")
                imputer = SimpleImputer(strategy="median")
                X_input = pd.DataFrame(imputer.fit_transform(X_input), columns=X_val.columns)
            else:
                raise e

        # 預測
        y_pred = model.predict(X_input)
        y_prob = model.predict_proba(X_input)[:, 1]

        # 指標
        acc = accuracy_score(y_input, y_pred)
        prec = precision_score(y_input, y_pred)
        rec = recall_score(y_input, y_pred)
        f1 = f1_score(y_input, y_pred)
        auc_val = roc_auc_score(y_input, y_prob)

        # ROC
        fpr, tpr, _ = roc_curve(y_input, y_prob)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0], tpr_interp[-1] = 0.0, 1.0

        fig_roc.add_trace(go.Scatter(
            x=mean_fpr, y=tpr_interp,
            mode='lines',
            name=f"{model_name} (AUC={auc_val:.3f})"
        ))

        # 混淆矩陣
        cm = confusion_matrix(y_input, y_pred)
        row, col = i // cols + 1, i % cols + 1
        fig_cm.add_trace(
            go.Heatmap(
                z=cm,
                x=["Predicted Negative", "Predicted Positive"],
                y=["Actual Negative", "Actual Positive"],
                colorscale='Blues',
                text=cm,
                texttemplate="%{text}",
                showscale=False
            ),
            row=row, col=col
        )

        results.append({
            "Model": model_name,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1 Score": f1,
            "ROC AUC": auc_val
        })

    # 隨機基準線
    fig_roc.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        line=dict(dash='dash'),
        name='Random Baseline'
    ))

    fig_roc.update_layout(
        title="ROC Curve Comparison",
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        width=800,
        height=600
    )
    fig_roc.show()

    fig_cm.update_layout(
        title="Confusion Matrices of All Models",
        width=400 * cols,
        height=300 * rows,
        showlegend=False
    )
    fig_cm.show()

    # 指標表格
    df_result = pd.DataFrame(results)

    return df_result

In [81]:
# 假設已經訓練完模型並存在 trained_models 中
evaluate_all_models_visual(trained_models, ex_X, ex_y)

🔍 Evaluating HybridXGBRF (Our Approach)...
🔍 Evaluating LogisticRegression (max_iter=200)...
🔍 Evaluating XGBClassifier...
🔍 Evaluating RandomForestClassifier...
🔍 Evaluating LogisticRegression (max_iter=1000)...
🔍 Evaluating Ridge...
🔍 Evaluating Lasso...
🔍 Evaluating Elastic...


,Model,Accuracy,Precision,Recall,F1 Score,ROC AUC
0,HybridXGBRF (Our Approach),0.798192,0.847484,0.295181,0.437855,0.829543
1,LogisticRegression (max_iter=200),0.798921,0.815233,0.316539,0.456016,0.789408
2,XGBClassifier,0.798338,0.842349,0.298467,0.440760,0.828575
3,RandomForestClassifier,0.778798,0.618755,0.440854,0.514870,0.792346
4,LogisticRegression (max_iter=1000),0.800671,0.822785,0.320372,0.461175,0.793582
5,Ridge,0.800671,0.827389,0.317634,0.459042,0.792244
6,Lasso,0.800671,0.826458,0.318182,0.459470,0.793281
7,Elastic,0.800671,0.827389,0.317634,0.459042,0.792756


In [82]:
# ===== 次族群實驗：年齡、ADL 變化、性別 =====
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def _first_existing_column(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def _make_sex_masks(df):
    """
    回傳 {'男性': mask, '女性': mask}；若無法判斷，回傳空 dict。
    支援：
      1) '性別' 欄位（值可能為 '男'/'女' 或 1/0/2…）
      2) one-hot 欄位 '性別_男' / '性別_女'
    """
    masks = {}
    if '性別_is_male' in df.columns:
        col = df['性別_is_male']
        # 嘗試各種常見標記
        male_mask = col.astype(str).str.contains('男') | (col == 1) | (col.astype(str).str.lower().isin(['m','male']))
        female_mask = col.astype(str).str.contains('女') | (col == 0) | (col.astype(str).str.lower().isin(['f','female']))
        if male_mask.any(): masks['男性'] = male_mask
        if female_mask.any(): masks['女性'] = female_mask
    else:
        male_col = _first_existing_column(df, ['性別_男','男','male','Male','M'])
        female_col = _first_existing_column(df, ['性別_女','女','female','Female','F'])
        if male_col is not None:
            masks['男性'] = df[male_col] == 1
        if female_col is not None:
            masks['女性'] = df[female_col] == 1
    return masks

def _make_adl_change_masks(df):
    """
    建立 ADL 變好/變差遮罩：
    'ADL_明顯惡化'（=0 視為變好，=1 變差）
    """
    masks = {}
    masks['ADL 變好'] = df['ADL_明顯惡化'] == 0
    masks['ADL 變差'] = df['ADL_明顯惡化'] == 1
    return masks

def _compute_metrics(y_true, y_prob, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1': f1_score(y_true, y_pred, zero_division=0),
        'ROC AUC': roc_auc_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else np.nan,
        'Support (n)': int(len(y_true)),
        'Positives (n)': int(y_true.sum())
    }

def evaluate_subgroups(models: dict, X_all: pd.DataFrame, y_all: pd.Series, raw_df_for_masks: pd.DataFrame):
    """
    models: 已訓練好的模型字典 trained_models
    X_all, y_all: 用於評估的特徵與標記（例如 ex_X, ex_y）
    raw_df_for_masks: 與 X_all 對齊、包含「年齡/ADL/性別」原始欄位的 DataFrame（例如 external）
    """
    # 年齡遮罩（需 '預估年齡'）
    subgroup_masks = {}
    if '預估年齡' in raw_df_for_masks.columns:
        subgroup_masks['年齡 > 85'] = raw_df_for_masks['預估年齡'] > 85
        subgroup_masks['年齡 <= 85'] = raw_df_for_masks['預估年齡'] <= 85
    else:
        print("⚠️ 找不到欄位『預估年齡』，跳過年齡分組。")

    # ADL 變化遮罩
    adl_masks = _make_adl_change_masks(raw_df_for_masks)
    if adl_masks:
        subgroup_masks.update(adl_masks)
    else:
        print("⚠️ 找不到可推算 ADL 變化的欄位，跳過 ADL 分組。")

    # 性別遮罩
    sex_masks = _make_sex_masks(raw_df_for_masks)
    if sex_masks:
        subgroup_masks.update(sex_masks)
    else:
        print("⚠️ 找不到可用的性別欄位，跳過性別分組。")

    rows = []
    for subgroup_name, mask in subgroup_masks.items():
        mask = mask.fillna(False).astype(bool)  # 安全轉型
        if mask.sum() == 0:
            print(f"⚠️ 次族群「{subgroup_name}」資料筆數為 0，略過。")
            continue

        X_sub = X_all.loc[mask]
        y_sub = y_all.loc[mask]

        # 若模型不支援 NaN，與你上面一致，統一補值策略（中位數）
        from sklearn.impute import SimpleImputer
        imputer = SimpleImputer(strategy="median")
        X_sub_imp = pd.DataFrame(imputer.fit_transform(X_sub), columns=X_sub.columns, index=X_sub.index)

        for model_name, model in models.items():
            # 預測
            y_prob = model.predict_proba(X_sub_imp)[:, 1]
            y_pred = (y_prob >= 0.5).astype(int)

            metrics = _compute_metrics(y_sub, y_prob, y_pred)
            metrics.update({'Subgroup': subgroup_name, 'Model': model_name})
            rows.append(metrics)

    result_df = pd.DataFrame(rows)
    if not result_df.empty:
        # 排序：先按次族群，再按 ROC AUC 由高到低
        result_df = result_df.sort_values(by=['Subgroup','ROC AUC'], ascending=[True, False]).reset_index(drop=True)
    return result_df

# === 執行：以外部驗證集為例 ===
subgroup_results = evaluate_subgroups(trained_models, ex_X, ex_y, external)
display(subgroup_results)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model
0,0.789509,0.842349,0.322295,0.466210,0.841792,5929,1691,ADL 變好,XGBClassifier
1,0.789341,0.847484,0.318746,0.463257,0.841564,5929,1691,ADL 變好,HybridXGBRF (Our Approach)
2,0.766740,0.618462,0.475458,0.537613,0.794722,5929,1691,ADL 變好,RandomForestClassifier
3,0.789678,0.830357,0.329982,0.472281,0.793475,5929,1691,ADL 變好,LogisticRegression (max_iter=1000)
4,0.789678,0.832335,0.328800,0.471386,0.793423,5929,1691,ADL 變好,Lasso
5,0.789678,0.832335,0.328800,0.471386,0.792959,5929,1691,ADL 變好,Elastic
6,0.789678,0.832335,0.328800,0.471386,0.792528,5929,1691,ADL 變好,Ridge
7,0.787991,0.823881,0.326434,0.467598,0.788248,5929,1691,ADL 變好,LogisticRegression (max_iter=200)
8,0.870829,0.692308,0.200000,0.310345,0.694710,929,135,ADL 變差,LogisticRegression (max_iter=1000)
9,0.870829,0.714286,0.185185,0.294118,0.692266,929,135,ADL 變差,Lasso


In [83]:
# 若只想看主力模型
display(subgroup_results[subgroup_results['Model'] == 'HybridXGBRF (Our Approach)'])

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model
1,0.789341,0.847484,0.318746,0.463257,0.841564,5929,1691,ADL 變好,HybridXGBRF (Our Approach)
14,0.854682,0.000000,0.000000,0.000000,0.673053,929,135,ADL 變差,HybridXGBRF (Our Approach)
16,0.833724,0.811111,0.297554,0.435388,0.822004,3416,736,女性,HybridXGBRF (Our Approach)
24,0.799958,0.834158,0.277138,0.416049,0.824157,4729,1216,年齡 <= 85,HybridXGBRF (Our Approach)
32,0.794270,0.870690,0.331148,0.479810,0.839343,2129,610,年齡 > 85,HybridXGBRF (Our Approach)
40,0.762929,0.874317,0.293578,0.439560,0.829271,3442,1090,男性,HybridXGBRF (Our Approach)


In [84]:
# ===== 複合次族群：年齡 × 性別 × ADL是否明顯惡化 =====
from itertools import product
import pandas as pd
from sklearn.impute import SimpleImputer

def evaluate_age_sex_composites(
    models: dict,
    X_all: pd.DataFrame,
    y_all: pd.Series,
    raw_df_for_masks: pd.DataFrame,
    age_threshold: int = 85,
    min_support: int = 10   # 次族群最少樣本數，太小就略過避免不穩定
):
    """
    針對「年齡(>threshold / <=threshold) × 性別(男性/女性) × ADL(變好/變差)」的交叉次族群做評估。
    會輸出每個模型在各交叉次族群的 Accuracy / Precision / Recall / F1 / ROC-AUC 等。
    依賴你已定義的: _make_sex_masks, _make_adl_change_masks, _compute_metrics。
    """
    if '預估年齡' not in raw_df_for_masks.columns:
        print("⚠️ 找不到欄位『預估年齡』，無法建立年齡遮罩。")
        return pd.DataFrame()

    # 年齡 bins
    age_masks = {
        f'年齡 > {age_threshold}': (raw_df_for_masks['預估年齡'] > age_threshold),
        f'年齡 <= {age_threshold}': (raw_df_for_masks['預估年齡'] <= age_threshold),
    }

    # 性別 masks（沿用你上面的 _make_sex_masks）
    sex_masks = _make_sex_masks(raw_df_for_masks)
    if not sex_masks:
        print("⚠️ 找不到可用的性別欄位，無法建立性別遮罩。")
        return pd.DataFrame()

    # ADL 變化 masks（沿用你上面的 _make_adl_change_masks）
    if 'ADL_明顯惡化' not in raw_df_for_masks.columns:
        print("⚠️ 找不到欄位『ADL_明顯惡化』，無法建立 ADL 遮罩。")
        return pd.DataFrame()
    adl_masks = _make_adl_change_masks(raw_df_for_masks)  # {'ADL 變好': mask, 'ADL 變差': mask}

    rows = []
    for (age_name, age_mask), (sex_name, sex_mask), (adl_name, adl_mask) in product(
        age_masks.items(), sex_masks.items(), adl_masks.items()
    ):
        combo_name = f"{age_name} & {sex_name} & {adl_name}"
        mask = (
            age_mask.fillna(False).astype(bool)
            & sex_mask.fillna(False).astype(bool)
            & adl_mask.fillna(False).astype(bool)
        )
        n = int(mask.sum())
        if n < min_support:
            print(f"ℹ️ 複合次族群「{combo_name}」樣本數 {n} < min_support={min_support}，略過。")
            continue

        X_sub = X_all.loc[mask]
        y_sub = y_all.loc[mask]

        # 與你現有策略一致：補缺失值（中位數）
        imputer = SimpleImputer(strategy="median")
        X_sub_imp = pd.DataFrame(imputer.fit_transform(X_sub), columns=X_sub.columns, index=X_sub.index)

        for model_name, model in models.items():
            y_prob = model.predict_proba(X_sub_imp)[:, 1]
            y_pred = (y_prob >= 0.5).astype(int)

            metrics = _compute_metrics(y_sub, y_prob, y_pred)
            metrics.update({'Subgroup': combo_name, 'Model': model_name})
            rows.append(metrics)

    result_df = pd.DataFrame(rows)
    if not result_df.empty:
        result_df['Prevalence'] = result_df['Positives (n)'] / result_df['Support (n)']
        result_df = result_df.sort_values(by=['Subgroup', 'ROC AUC'], ascending=[True, False]).reset_index(drop=True)
    return result_df

In [85]:
age_sex_results = evaluate_age_sex_composites(
    trained_models, ex_X, ex_y, external,
    age_threshold=85,
    min_support=10
)
display(age_sex_results)

pivot_auc = age_sex_results.pivot_table(index='Subgroup', columns='Model', values='ROC AUC')
display(pivot_auc)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.840871,0.805755,0.302703,0.440079,0.838543,1791,370,年齡 <= 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.206588
1,0.841429,0.802817,0.308108,0.445312,0.835728,1791,370,年齡 <= 85 & 女性 & ADL 變好,XGBClassifier,0.206588
2,0.803462,0.531250,0.413514,0.465046,0.795263,1791,370,年齡 <= 85 & 女性 & ADL 變好,RandomForestClassifier,0.206588
3,0.839196,0.780822,0.308108,0.441860,0.777757,1791,370,年齡 <= 85 & 女性 & ADL 變好,LogisticRegression (max_iter=200),0.206588
4,0.839754,0.786207,0.308108,0.442718,0.772371,1791,370,年齡 <= 85 & 女性 & ADL 變好,Lasso,0.206588
...,...,...,...,...,...,...,...,...,...,...
59,0.867769,0.875000,0.318182,0.466667,0.707530,121,22,年齡 > 85 & 男性 & ADL 變差,Elastic,0.181818
60,0.876033,0.888889,0.363636,0.516129,0.704775,121,22,年齡 > 85 & 男性 & ADL 變差,LogisticRegression (max_iter=200),0.181818
61,0.867769,0.875000,0.318182,0.466667,0.704316,121,22,年齡 > 85 & 男性 & ADL 變差,Ridge,0.181818
62,0.818182,0.000000,0.000000,0.000000,0.679752,121,22,年齡 > 85 & 男性 & ADL 變差,HybridXGBRF (Our Approach),0.181818


Model,Elastic,HybridXGBRF (Our Approach),Lasso,LogisticRegression (max_iter=1000),LogisticRegression (max_iter=200),RandomForestClassifier,Ridge,XGBClassifier
Subgroup,,,,,,,,
年齡 <= 85 & 女性 & ADL 變好,0.771595,0.838543,0.772371,0.772173,0.777757,0.795263,0.770895,0.835728
年齡 <= 85 & 女性 & ADL 變差,0.649274,0.681640,0.651024,0.653803,0.651127,0.698055,0.646187,0.684265
年齡 <= 85 & 男性 & ADL 變好,0.783655,0.833461,0.784204,0.784452,0.769163,0.784467,0.783201,0.836006
年齡 <= 85 & 男性 & ADL 變差,0.713603,0.654380,0.715926,0.718713,0.705839,0.680292,0.712541,0.608892
年齡 > 85 & 女性 & ADL 變好,0.766013,0.825553,0.766249,0.766509,0.772011,0.778942,0.765850,0.826378
年齡 > 85 & 女性 & ADL 變差,0.679668,0.706362,0.683184,0.684143,0.673593,0.664802,0.675192,0.657928
年齡 > 85 & 男性 & ADL 變好,0.803810,0.855666,0.804316,0.804307,0.799626,0.802792,0.803323,0.853550
年齡 > 85 & 男性 & ADL 變差,0.707530,0.679752,0.708907,0.713039,0.704775,0.738522,0.704316,0.638659


In [86]:
# 每個模型在哪個次族群表現最好（同樣以 ROC AUC 為主）
def best_subgroup_per_model(results_df: pd.DataFrame,
                            primary='ROC AUC',
                            tie_breakers=('F1','Recall','Precision','Accuracy','Support (n)')):
    sort_cols = ['Model', primary, *tie_breakers]
    sort_asc  = [True, False, *([False]*len(tie_breakers))]
    df_sorted = results_df.sort_values(by=sort_cols, ascending=sort_asc)
    best_df = df_sorted.groupby('Model', as_index=False).head(1).reset_index(drop=True)
    return best_df

best_subgroup_each_model = best_subgroup_per_model(age_sex_results)
display(best_subgroup_each_model)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.738275,0.916667,0.376426,0.533693,0.803810,661,263,年齡 > 85 & 男性 & ADL 變好,Elastic,0.397882
1,0.736762,0.940594,0.361217,0.521978,0.855666,661,263,年齡 > 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.397882
2,0.738275,0.916667,0.376426,0.533693,0.804316,661,263,年齡 > 85 & 男性 & ADL 變好,Lasso,0.397882
3,0.738275,0.916667,0.376426,0.533693,0.804307,661,263,年齡 > 85 & 男性 & ADL 變好,LogisticRegression (max_iter=1000),0.397882
4,0.735250,0.915094,0.368821,0.525745,0.799626,661,263,年齡 > 85 & 男性 & ADL 變好,LogisticRegression (max_iter=200),0.397882
5,0.739788,0.707763,0.589354,0.643154,0.802792,661,263,年齡 > 85 & 男性 & ADL 變好,RandomForestClassifier,0.397882
6,0.738275,0.916667,0.376426,0.533693,0.803323,661,263,年齡 > 85 & 男性 & ADL 變好,Ridge,0.397882
7,0.735250,0.931373,0.361217,0.520548,0.853550,661,263,年齡 > 85 & 男性 & ADL 變好,XGBClassifier,0.397882


In [87]:
# 直接列出「整體表現最高的 (Subgroup, Model) Top-K」
def top_k_overall(results_df: pd.DataFrame, k=10,
                  primary='ROC AUC',
                  tie_breakers=('F1','Recall','Precision','Accuracy','Support (n)')):
    sort_cols = [primary, *tie_breakers]
    sort_asc  = [False, *([False]*len(tie_breakers))]
    return results_df.sort_values(by=sort_cols, ascending=sort_asc).head(k).reset_index(drop=True)

top10 = top_k_overall(age_sex_results, k=10)
display(top10)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.736762,0.940594,0.361217,0.521978,0.855666,661,263,年齡 > 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.397882
1,0.735250,0.931373,0.361217,0.520548,0.853550,661,263,年齡 > 85 & 男性 & ADL 變好,XGBClassifier,0.397882
2,0.840871,0.805755,0.302703,0.440079,0.838543,1791,370,年齡 <= 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.206588
3,0.758044,0.849624,0.301333,0.444882,0.836006,2331,750,年齡 <= 85 & 男性 & ADL 變好,XGBClassifier,0.321750
4,0.841429,0.802817,0.308108,0.445312,0.835728,1791,370,年齡 <= 85 & 女性 & ADL 變好,XGBClassifier,0.206588
5,0.757615,0.849057,0.300000,0.443350,0.833461,2331,750,年齡 <= 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.321750
6,0.803665,0.802920,0.357143,0.494382,0.826378,1146,308,年齡 > 85 & 女性 & ADL 變好,XGBClassifier,0.268761
7,0.803665,0.816794,0.347403,0.487472,0.825553,1146,308,年齡 > 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.268761
8,0.738275,0.916667,0.376426,0.533693,0.804316,661,263,年齡 > 85 & 男性 & ADL 變好,Lasso,0.397882
9,0.738275,0.916667,0.376426,0.533693,0.804307,661,263,年齡 > 85 & 男性 & ADL 變好,LogisticRegression (max_iter=1000),0.397882


In [88]:
# 每個次族群下，表現最好的模型（以 ROC AUC 為主，F1/Recall/Precision/Accuracy/Support 作為平手時的次序）
def best_model_per_subgroup(results_df: pd.DataFrame,
                            primary='ROC AUC',
                            tie_breakers=('F1','Recall','Precision','Accuracy','Support (n)')):
    sort_cols = ['Subgroup', primary, *tie_breakers]
    sort_asc  = [True, False, *([False]*len(tie_breakers))]
    df_sorted = results_df.sort_values(by=sort_cols, ascending=sort_asc)
    # 取每個 Subgroup 的第一列（即最佳模型）
    best_df = df_sorted.groupby('Subgroup', as_index=False).head(1).reset_index(drop=True)
    return best_df

best_by_subgroup = best_model_per_subgroup(age_sex_results)
display(best_by_subgroup)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.840871,0.805755,0.302703,0.440079,0.838543,1791,370,年齡 <= 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.206588
1,0.852518,0.500000,0.024390,0.046512,0.698055,278,41,年齡 <= 85 & 女性 & ADL 變差,RandomForestClassifier,0.147482
2,0.758044,0.849624,0.301333,0.444882,0.836006,2331,750,年齡 <= 85 & 男性 & ADL 變好,XGBClassifier,0.321750
3,0.854103,0.705882,0.218182,0.333333,0.718713,329,55,年齡 <= 85 & 男性 & ADL 變差,LogisticRegression (max_iter=1000),0.167173
4,0.803665,0.802920,0.357143,0.494382,0.826378,1146,308,年齡 > 85 & 女性 & ADL 變好,XGBClassifier,0.268761
5,0.915423,0.000000,0.000000,0.000000,0.706362,201,17,年齡 > 85 & 女性 & ADL 變差,HybridXGBRF (Our Approach),0.084577
6,0.736762,0.940594,0.361217,0.521978,0.855666,661,263,年齡 > 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.397882
7,0.826446,1.000000,0.045455,0.086957,0.738522,121,22,年齡 > 85 & 男性 & ADL 變差,RandomForestClassifier,0.181818
